In [1]:
import numpy as np
import cv2
import cv2.aruco as aruco
import pafy

In [2]:
#使用ChArUco Board做相機校正
cap = cv2.VideoCapture('CharUco_board.mp4')

totalFrame   = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) #獲取影片總幀數
frameWidth   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))//2 #獲取影片一半大小的寬
frameHeight  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))//2 #獲取影片一半大小的高

arucoParams  = aruco.DetectorParameters_create()
#創建 aruco 檢測器參數對象，可以用來設置偵測器的參數
arucoParams.cornerRefinementMethod = aruco.CORNER_REFINE_SUBPIX
#使用子像素級別的角點檢測演算法，以提高角點檢測的準確性
arucoDict    = aruco.Dictionary_get(aruco.DICT_6X6_250)
#加載了預定義的ArUco標記字典。6x6表示每個標記都是一個6x6的方陣
#250表示這個字典中一共包含了250個這樣的標記，每個標記都有一個唯一的ID

# 必須描述ChArUco board的尺寸規格
gridX        = 5 # 水平方向5格
gridY        = 7 # 垂直方向7格
squareSize   = 4 # 每格為4cmX4cm
charucoBoard = aruco.CharucoBoard_create(gridX,gridY,squareSize,squareSize/2,arucoDict)
#gridX x gridY 的 ChArUco board
#squareLength:每個格子的邊長
#markerLength:示每個 ArUco marker 的邊長 -> ArUco marker為2cmX2cm
#arucoDict:要使用的 ArUco 字典

print('height {}, width {}'.format(cap.get(cv2.CAP_PROP_FRAME_HEIGHT),cap.get(cv2.CAP_PROP_FRAME_WIDTH)))
refinedStrategy = True #refinedStrategy 表示是否启用检测到的角点的精确化方法
criteria        = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.00001)
#criteria 是用來控制這個迭代(求解相機內部參數時)過程的停止條件
#當最大迭代次數(cv2.TERM_CRITERIA_MAX_ITER)為100，收斂誤差小(cv2.TERM_CRITERIA_EPS)於0.00001時，就停止迭代求解內部參數
frameId        = 0
collectCorners = []
collectIds     = []
collectFrames  = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
        
    frame = cv2.resize(frame,(frameWidth,frameHeight)) #重塑影像大小
    (corners, ids, rejected) = aruco.detectMarkers(frame, arucoDict, parameters=arucoParams)  
    #aruco.detectMarkers 函數偵測影像中的 ArUco marker
    #傳入影格、aruco 字典和偵測參數 arucoParams 作為參數
    #回傳檢測到的標記的角落點位置（corners）、對應的標記 ID（ids），以及被排除的標記（rejected）
    #角落點位置是一個 N 個元素的 List，每個元素包含標記上的角落點位置
    #標記 ID 則是一個 N 個元素的 List，每個元素是標記的整數 ID
    #被排除的標記則是沒有被偵測到的標記
    
    if refinedStrategy:
        corners, ids, _, _ = aruco.refineDetectedMarkers(frame,charucoBoard,corners,ids,rejected)
        #aruco.refineDetectedMarkers() 函數進行檢測到的 ArUco marker 的角點精確化
        #frame：當前的影像幀。
        #charucoBoard：包含所檢測的 ArUco marker 的 CharUco board。
        #corners：所有檢測到的 ArUco marker 的角點。
        #ids：所有檢測到的 ArUco marker 的 ID。
        #rejected：被拒絕的角點。
        #輸出為精確化後的角點、ID，以及被拒絕的角點
    if frameId % 100 == 50 and len(ids)==17: #當幀數(frameId)除以100的餘數為50，且檢測到的ArUco markers數量為17時
        #收集符合條件的角點(corners)、ID(ids)、以及圖像幀(frame)，並將它們儲存在列表中
        collectCorners.append(corners)
        collectIds.append(ids.ravel()) #ravel:用於將多維數組降為一維
        collectFrames.append(frame)
        
    if len(corners) > 0: #偵測到一組以上的ArUco marker
        aruco.drawDetectedMarkers(frame, corners, ids) #會在影像上繪製偵測到的 ArUco 標記的邊界框和標記 ID

    cv2.imshow('Analysis of a CharUco board for camera calibration',frame)
    if cv2.waitKey(20) != -1:
        break
        
    frameId += 1

cv2.destroyAllWindows()
cap.release()

height 1080.0, width 1920.0


In [3]:
#使用calibrateCameraAruco
caliCorners=np.concatenate([np.array(x).reshape(-1,4,2) for x in collectCorners],axis=0)
#list轉numpy，並reshape成三維數組。-1 表示根據數組長度自動填缺失的那個維度大小，也就是角點組數量
#numpy.concatenate用於將兩個或多個數組沿指定軸連接在一起，其中axis=0表沿著角點組數量拼接
counter=np.array([len(x) for x in collectIds])
#counter 的長度應該等於 len(collectIds)，也就是50幀內共幾幀捕捉到17個ArUco markers
#內容皆為17(len(x))，因為棋盤中有17個ArUco markers
caliIds=np.array(collectIds).ravel()
#collectIds種成numpy再拉平成一維數組
cameraMatrixInit = np.array([[ 1000.,    0., frameWidth/2.],[    0., 1000., frameHeight/2.],[    0.,    0.,           1.]])
#相機內參初值
distCoeffsInit   = np.zeros((5,1))
#相機畸變係數初值
ret, aruco_cameraMatrix, aruco_distCoeffs, aruco_rvects, aruco_tvects = aruco.calibrateCameraAruco(caliCorners,caliIds,counter,charucoBoard,(frameWidth,frameHeight),cameraMatrixInit,distCoeffsInit)
#aruco.calibrateCameraAruco() 函数，將收集到的所資料傳遞給 ArUco 相機校正函數，以獲得最終的相機校正參數
#校正後內參矩陣(aruco_cameraMatrix)、校正後畸變係數(aruco_distCoeffs)、旋轉向量(aruco_rvects)、平移向量(aruco_tvects)
print(aruco_cameraMatrix)
print(aruco_distCoeffs)
print(aruco_rvects)
print(aruco_tvects)

[[913.81070779   0.         479.5586767 ]
 [  0.         921.0914589  293.00945991]
 [  0.           0.           1.        ]]
[[ 0.06739156]
 [-0.11176159]
 [-0.00446193]
 [-0.00266387]
 [-0.01974274]]
(array([[ 1.85560181],
       [ 2.13559441],
       [-0.43460999]]), array([[ 2.02379095],
       [ 2.16232636],
       [-0.37108907]]), array([[ 1.95358902],
       [ 2.13934218],
       [-0.43973432]]), array([[ 2.07453931],
       [ 2.14265824],
       [-0.2240849 ]]), array([[ 1.84966146],
       [ 2.40870897],
       [-0.12752218]]), array([[ 2.00997394],
       [ 2.25545981],
       [-0.31581337]]), array([[ 2.03659511],
       [ 2.15591552],
       [-0.52993239]]), array([[ 2.12746156],
       [ 2.19254096],
       [-0.09196582]]), array([[ 2.22641063],
       [ 2.13735615],
       [-0.11921934]]), array([[ 2.23525175],
       [ 2.03372319],
       [-0.02535581]]), array([[ 2.23620076],
       [ 2.09170461],
       [-0.1405798 ]]))
(array([[-12.14589974],
       [-10.64405907],
 

In [4]:
#使用calibriateCameraChAruco
caliCorners=[]
caliIds    =[]
for corners, ids, frame in zip(collectCorners,collectIds,collectFrames):
    ret, charucoCorners, charucoIds = aruco.interpolateCornersCharuco(corners,ids,frame,charucoBoard,aruco_cameraMatrix,aruco_distCoeffs)
    #interpolateCornersCharuco:用於Charuco棋盤格，用於提取Charuco棋盤格上的所有角點和標記的ID
    #類似detectMarkers的功用，但拿到的訊息更多，所以可以提高相機校正的準度
    caliCorners.append(charucoCorners)
    caliIds.append(charucoIds)

ret, charuco_cameraMatrix, charuco_distCoeffs, charuco_rvects, charuco_tvects = aruco.calibrateCameraCharuco(caliCorners,caliIds,charucoBoard,(frameWidth,frameHeight), aruco_cameraMatrix,aruco_distCoeffs)    
print(charuco_cameraMatrix)
print(charuco_distCoeffs)
print(charuco_rvects)
print(charuco_tvects)

[[888.25457479   0.         482.34262359]
 [  0.         896.25270996 305.57029901]
 [  0.           0.           1.        ]]
[[ 0.08518861]
 [-0.41241448]
 [-0.00235481]
 [-0.00228344]
 [ 1.0810133 ]]
(array([[ 1.86281848],
       [ 2.14235737],
       [-0.42323125]]), array([[ 2.03014216],
       [ 2.16949307],
       [-0.36027035]]), array([[ 1.96162848],
       [ 2.14740957],
       [-0.43240375]]), array([[ 2.0803946 ],
       [ 2.14909494],
       [-0.21471339]]), array([[ 1.84935369],
       [ 2.40862702],
       [-0.12541285]]), array([[ 2.01467574],
       [ 2.26127206],
       [-0.30519933]]), array([[ 2.04292273],
       [ 2.16299229],
       [-0.51617167]]), array([[ 2.13256338],
       [ 2.19833919],
       [-0.08449552]]), array([[ 2.23040726],
       [ 2.1419325 ],
       [-0.11295605]]), array([[ 2.23796388],
       [ 2.03550134],
       [-0.0238541 ]]), array([[ 2.24034167],
       [ 2.0957615 ],
       [-0.13390518]]))
(array([[-12.28808103],
       [-11.36223488],
 

In [5]:
#抓影片
urls = [
    "https://youtu.be/CWp2ZDMWn2g",
    "https://youtu.be/BfyH4ROtFm0",
    "https://youtu.be/o1WXFxHXieM",
    "https://youtu.be/lzyDD8bMDKs",
    "https://youtu.be/Zs1sHB6DNR4",
    "https://youtu.be/v2HN4gd66nM"
]
videos = []
# 遍歷6個url，依次獲取最佳解析度和編碼格式，並創建影片對象
for url in urls:
    video = pafy.new(url, basic=False, gdata=False) #創建pafy對象
    #basic：如果設置為True，則只獲取基本元數據（如影片標題、作者、影片ID等）
    #basic：如果設置為False，還會獲取更詳細的數據（如影片描述、上傳日期、喜歡和不喜歡的數量、影片解析度、時長等）
    #gdata：如果設置為True，則從gdata API(google提供的，但只要用yt的話，YouTube API會較方便)獲取數據，否則從YouTube API獲取數據
    best = video.getbest(preftype="mp4") #獲取最佳的影片解析度和編碼格式
    cap_yt = cv2.VideoCapture(best.url)
    videos.append(cap_yt)

In [12]:
#2d
cap = cv2.VideoCapture('arUco_marker.mp4')
markerSize  = 6 #6cm

frameWidth   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))//2
frameHeight  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))//2

arucoParams = aruco.DetectorParameters_create()
arucoParams.cornerRefinementMethod = aruco.CORNER_REFINE_SUBPIX
arucoDict   = aruco.Dictionary_get(aruco.DICT_7X7_50)

print('height {}, width {}'.format(cap.get(cv2.CAP_PROP_FRAME_HEIGHT),cap.get(cv2.CAP_PROP_FRAME_WIDTH)))

while True:
    ret, frame = cap.read()
    if not ret:
        break
           
    frame = cv2.resize(frame,(frameWidth,frameHeight)) 
    # 將圖像轉換為灰度圖像並進行二值化處理
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 51, 1)
    #自適應二值化，使用到高斯濾波去除圖像中的噪聲，提高算法的準確性
    #255:當像素值超過閾值時，將被賦予的值
    #cv2.ADAPTIVE_THRESH_GAUSSIAN_C:自適應方法，對光照變化更好
    #cv2.THRESH_BINARY:二值化類型
    #25:指定區域的大小，必須為奇數
    #1:從均值或加權平均值中減去的常數值
    (corners, ids, rejected) = aruco.detectMarkers(thresh, arucoDict, parameters=arucoParams,cameraMatrix=charuco_cameraMatrix, distCoeff=charuco_distCoeffs)   
    #加了校正後的參數進行ArUco marker檢測
    if len(corners) > 0:
        aruco.drawDetectedMarkers(frame, corners, ids)
                
        for i, corner in enumerate(corners):
            for pt in corner[0]:
                cv2.drawMarker(frame,tuple(pt.astype(int).ravel()),(0,0,255),cv2.MARKER_CROSS,20,2)
                #在每個標記的四個角上繪制紅色的十字形標記
            
            idd = ids[i] #拿到當前corner的id
            ret, yt_frame = videos[int(idd)-1].read() #-1是因為videos從0開始，corners從1開始
            if ret == False: #影片沒了就跳出
                break
            wid = yt_frame.shape[1]
            high = yt_frame.shape[0]
            
            pts = np.array(corner[0], np.float32) #拿到一組四個角點
            #製造四個角點從左上方開始順時鐘排列
            # 對數組進行排序，先排y再排x
            pts = pts[np.argsort(pts[:, 1])] #照pts中的第二元素升序排列
            if pts[0][0] > pts[1][0]:
                pts_copy = pts.copy()
                pts[0], pts[1] = pts_copy[1], pts_copy[0] #互換
            if pts[2][0] < pts[3][0]:
                pts_copy = pts.copy()
                pts[2], pts[3] = pts_copy[3], pts_copy[2]
            
            #獲得要貼過去的yt影片角點，從左上方開始順時鐘排列
            srcp=np.array([[0, 0], [wid, 0], [wid, high], [0, high]]).astype('float32')

            # 計算兩組4個點之間的透視變換矩陣（3x3）
            M = cv2.getPerspectiveTransform(srcp, pts)
            #圖像幾何變換函數，用於實現透視變換
            warped_img = cv2.warpPerspective(yt_frame,M, frame.shape[:2][::-1])#參數:待處理的原始圖像、變換矩陣M、變換後圖像的大小
            #取代目標圖中的不規則四邊形區域
            #做出一個遮罩，在需要被換掉的像素填上255，其餘為0
            mask = np.zeros(frame.shape[:2], dtype=np.uint8)
            cv2.fillPoly(mask, [pts.astype(np.int32)], 255)
            frame[mask != 0] = warped_img[mask != 0] #mask中不為0的像素換成yt影片
            
        rvects, tvects, _ = aruco.estimatePoseSingleMarkers(corners, markerSize, charuco_cameraMatrix, charuco_distCoeffs)
        #從檢測到的單個標記（marker）的角點坐標計算其相對於相機坐標系的旋轉向量和平移向量
        #corners是檢測到的角點坐標
        #Marker尺寸
        #cameraMatrixInit是相機的內參矩陣
        #distCoeffsInit是相機的畸變參數
        for rvec,tvec in zip(rvects,tvects):
            aruco.drawAxis(frame, charuco_cameraMatrix, charuco_distCoeffs, rvec, tvec, markerSize/2)
            #繪製標記的坐標系，以可視化其方向
    cv2.imshow('for Q1 and Q2',frame)
    if cv2.waitKey(20) != -1:
        break

cv2.destroyAllWindows()
cap.release()

height 1080.0, width 1920.0


In [6]:
#3d
cap = cv2.VideoCapture('arUco_marker.mp4')
markerSize  = 6 #6cm

frameWidth   = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))//2
frameHeight  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))//2

arucoParams = aruco.DetectorParameters_create()
arucoParams.cornerRefinementMethod = aruco.CORNER_REFINE_SUBPIX
arucoDict   = aruco.Dictionary_get(aruco.DICT_7X7_50)

print('height {}, width {}'.format(cap.get(cv2.CAP_PROP_FRAME_HEIGHT),cap.get(cv2.CAP_PROP_FRAME_WIDTH)))
markerCorners3D = np.array([[3,0,0],[-3,0,0],[-3,-3,6],[3,-3,6]],dtype=float) #調整顯示的座標
while True:
    ret, frame = cap.read()
    if not ret:
        break
           
    frame = cv2.resize(frame,(frameWidth,frameHeight)) 
    # 將圖像轉換為灰度圖像並進行二值化處理
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 51, 1)
    #自適應二值化，使用到高斯濾波去除圖像中的噪聲，提高算法的準確性
    #255:當像素值超過閾值時，將被賦予的值
    #cv2.ADAPTIVE_THRESH_GAUSSIAN_C:自適應方法，對光照變化更好
    #cv2.THRESH_BINARY:二值化類型
    #25:指定區域的大小，必須為奇數
    #1:從均值或加權平均值中減去的常數值
    (corners, ids, rejected) = aruco.detectMarkers(thresh, arucoDict, parameters=arucoParams,cameraMatrix=charuco_cameraMatrix, distCoeff=charuco_distCoeffs)   
    #加了校正後的參數進行ArUco marker檢測
    if len(corners) > 0:
            
        rvects, tvects, _ = aruco.estimatePoseSingleMarkers(corners, markerSize, charuco_cameraMatrix, charuco_distCoeffs)
        #從檢測到的單個標記（marker）的角點坐標計算其相對於相機坐標系的旋轉向量和平移向量
        #corners是檢測到的角點坐標
        #Marker尺寸
        #cameraMatrixInit是相機的內參矩陣
        #distCoeffsInit是相機的畸變參數
        for i, (rvec, tvec) in enumerate(zip(rvects, tvects)):
            proj_pt_with_dist,_ = cv2.projectPoints(markerCorners3D, rvec, tvec, charuco_cameraMatrix, charuco_distCoeffs)
            #使用 cv2.projectPoints() 函數將每個標記的三維座標投影到二維圖像平面上
            
            idd = ids[i] #拿到當前corner的id
            ret, yt_frame = videos[int(idd)-1].read() #-1是因為videos從0開始，corners從1開始
            if ret == False: #影片沒了就跳出
                break
            wid = yt_frame.shape[1]
            high = yt_frame.shape[0]

            pts = np.array(proj_pt_with_dist, np.float32)
            pts = pts.reshape((4, 2)) #拿到一組四個角點
            #製造四個角點從左上方開始順時鐘排列
            # 對數組進行排序，先排y再排x
            pts = pts[np.argsort(pts[:, 1])] #照pts中的第二元素升序排列
            if pts[0][0] > pts[1][0]:
                pts_copy = pts.copy()
                pts[0], pts[1] = pts_copy[1], pts_copy[0] #互換
            if pts[2][0] < pts[3][0]:
                pts_copy = pts.copy()
                pts[2], pts[3] = pts_copy[3], pts_copy[2]
            
            #獲得要貼過去的yt影片角點，從左上方開始順時鐘排列
            srcp=np.array([[0, 0], [wid, 0], [wid, high], [0, high]]).astype('float32')

            # 計算兩組4個點之間的透視變換矩陣（3x3）
            M = cv2.getPerspectiveTransform(srcp, pts)
            #圖像幾何變換函數，用於實現透視變換
            warped_img = cv2.warpPerspective(yt_frame,M, frame.shape[:2][::-1])#參數:待處理的原始圖像、變換矩陣M、變換後圖像的大小
            #取代目標圖中的不規則四邊形區域
            #做出一個遮罩，在需要被換掉的像素填上255，其餘為0
            mask = np.zeros(frame.shape[:2], dtype=np.uint8)
            cv2.fillPoly(mask, [pts.astype(np.int32)], 255)
            frame[mask != 0] = warped_img[mask != 0] #mask中不為0的像素換成yt影片
    cv2.imshow('for Q1 and Q2',frame)
    if cv2.waitKey(20) != -1:
        break

cv2.destroyAllWindows()
cap.release()

height 1080.0, width 1920.0
